In [13]:
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np

import sys
!{sys.executable} -m pip install numba

df = pd.read_csv("../data/battery_data.csv", index_col=0, parse_dates=True)

# Créer un réseau vide, une toile vierge pour notre micro-réseau
net = pp.create_empty_network()

# Bus principal / Slack (réseau)
bus_slack = pp.create_bus(net, vn_kv=20.0, name="Bus Slack") # Bus de référence pour le réseau, tension nominale de 20 kV
pp.create_ext_grid(net, bus=bus_slack, vm_pu=1.0, name="Source Réseau") # Source d'alimentation du réseau, connectée au bus de référence

# Bus PV
bus_pv = pp.create_bus(net, vn_kv=0.4, name="Bus PV") # Bus pour la production photovoltaïque, tension nominale de 0.4 kV

#Bus batterie
bus_batt = pp.create_bus(net, vn_kv=0.4, name="Bus Batterie") # Bus pour la batterie, tension nominale de 0.4 kV

# Bus bâtiment
bus_load = pp.create_bus(net, vn_kv=0.4, name="Bus Load") # Bus pour la consommation du bâtiment, tension nominale de 0.4 kV

#Transformateur 20/0.4 kV
pp.create_transformer_from_parameters(net, hv_bus=bus_slack, lv_bus=bus_load, sn_mva=25, # puissance nominale de 25 MVA
                                      vn_hv_kv=20, # tension nominale côté haute tension de 20 kV
                                      vn_lv_kv=0.4, # tension nominale côté basse tension de 0.4 kV
                                      vk_percent=10,     # tension de court-circuit
                                      vkr_percent=0.5,   # résistance
                                      pfe_kw=0,          # pertes à vide
                                      i0_percent=0,      # courant à vide
                                      name="Trafo 20/0.4 kV")

# Ligne entre bus
pp.create_line(net, bus_load, bus_pv, length_km=0.05, std_type="NAYY 4x50 SE") # Ligne entre le bus de charge et le bus PV, std_type correspond au type de câble utilisé
pp.create_line(net, bus_load, bus_batt, length_km=0.05, std_type="NAYY 4x50 SE") # Ligne entre le bus de charge et le bus batterie


[notice] A new release of pip is available: 25.3 -> 26.0.1

[notice] To update, run: python.exe -m pip install --upgrade pip

np.int64(1)

In [14]:
# Création des éléments actifs du réseau
pp.create_sgen(net, bus=bus_pv, p_mw=0.2, q_mvar=0, name="PV Plant") # Générateur statique pour la production photovoltaïque, puissance de 200 kW
pp.create_storage(net, bus=bus_batt, p_mw=0.15, max_e_mwh=0.3, min_e_mwh=0, name="Battery", efficiency_percent=90) # Stockage pour la batterie, puissance de 150 kW, capacité de 300 kWh, efficacité de 90%
pp.create_load(net, bus=bus_load, p_mw=0.5, q_mvar=0, name="Load Building") # Charge pour la consommation du bâtiment, puissance de 500 kW

print(net.bus)

           name  vn_kv type  zone  in_service   geo
0     Bus Slack   20.0    b  None        True  None
1        Bus PV    0.4    b  None        True  None
2  Bus Batterie    0.4    b  None        True  None
3      Bus Load    0.4    b  None        True  None


In [15]:
# Simulation de l'état du réseau, calcul des flux de puissance, tensions, etc.
pp.runpp(net)
print(net.res_bus) # Affiche les résultats de la simulation pour les bus (tensions, angles, etc.)
print(net.res_line) # Affiche les résultats de la simulation pour les lignes (courants, pertes, etc.)

      vm_pu  va_degree      p_mw    q_mvar
0  1.000000   0.000000 -0.462293 -0.002436
1  1.038524   0.180423 -0.200000 -0.000000
2  0.968829  -0.335915  0.150000  0.000000
3  0.999900  -0.105800  0.500000  0.000000
   p_from_mw  q_from_mvar  p_to_mw     q_to_mvar     pl_mw   ql_mvar  \
0  -0.192559     0.000961     0.20  3.035417e-12  0.007441  0.000961   
1   0.154809     0.000621    -0.15  5.899366e-13  0.004809  0.000621   

   i_from_ka   i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  \
0   0.277967  0.277967  0.277967      0.9999         -0.1058  1.038524   
1   0.223472  0.223472  0.223472      0.9999         -0.1058  0.968829   

   va_to_degree  loading_percent  
0      0.180423       195.751299  
1     -0.335915       157.374712  


In [16]:
# Simulation sur l'ensemble de la période, en mettant à jour les injections de puissance à chaque pas de temps
for t, row in df.iterrows():
    net.sgen.at[0, 'p_mw'] = row['pv_kW'] / 1000  # Convertir kW en MW
    net.load.at[0, 'p_mw'] = row['power_kW'] / 1000  # Convertir kW en MW
   
    pp.runpp(net)

    df.loc[t, 'v_bus_load'] = net.res_bus.vm_pu[bus_load] # Tension au bus de charge après la simulation
    df.loc[t, "import_grid"] = net.res_ext_grid.p_mw[0] * 1000 # Importation depuis le réseau en kW
    df.head()